In [ ]:
pip install selenium

Setting Up Selenium And Logging into Youtube

In [ ]:
import time
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from dotenv import dotenv_values


config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")

email = config["Email"]
password = config["Password"]

driver = webdriver.Firefox()


try:
   # Open YouTube
    driver.get("https://www.youtube.com")
    time.sleep(3)  # Allow time for the page to load

    # Click on the "Sign In" button
    sign_in_button = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Sign in']")
    sign_in_button.click()
    time.sleep(1)  # Wait for the login page to load

    # Enter email
    email_field = driver.find_element(By.ID, "identifierId")
    email_field.send_keys(email)
    email_field.send_keys(Keys.RETURN)
    time.sleep(1)  # Wait for the password page to load

    # Enter password
    password_field = driver.find_element(By.NAME, "Passwd")
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)
    time.sleep(1)  # Allow time for login to complete


    # Verify if login is successful
    if "youtube.com" in driver.current_url:
        print("Login successful!")
    else:
        print("Login might have failed. Please check manually.")
finally:
   print("DONE!")



To scrap Youtube URLs

In [ ]:
from selenium.common.exceptions import StaleElementReferenceException
def apply_filters(driver):
    """
    Apply YouTube filters for duration (4-20 minutes) and subtitles/CC.

    Args:
        driver (webdriver): Selenium WebDriver instance.
    """
    # Click on the 'Filter' button
    filter_button = driver.find_element(By.ID, "filter-button")
    filter_button.click()
    time.sleep(2)

    # Select '4-20 minutes' under duration
    duration_filter = driver.find_element(By.XPATH, "//yt-formatted-string[text()='4 - 20 minutes']")
    duration_filter.click()
    time.sleep(2)

     # Click on the 'Filter' button
    filter_button = driver.find_element(By.ID, "filter-button")
    filter_button.click()
    time.sleep(2)

    # Select 'Subtitles/CC' filter using its title attribute
    subtitles_filter = driver.find_element(By.XPATH, "//div[@title='Search for Subtitles/CC']")
    subtitles_filter.click()
    time.sleep(2)

def get_youtube_video_urls1(driver, search_topic, max_urls):
    """
    Fetch YouTube video URLs for a given search topic.

    Args:
        driver (webdriver): Selenium WebDriver instance.
        search_topic (str): The topic to search on YouTube.
        max_urls (int): Maximum number of video URLs to collect.

    Returns:
        list: A list of collected YouTube video URLs.
    """
    # Search for the given topic
    search_box = driver.find_element(By.NAME, "search_query")
    search_box.clear()
    search_box.send_keys(search_topic)
    search_box.submit()
    time.sleep(5)  # Wait for results to load

    apply_filters(driver)

    # Scroll and collect video URLs
    video_urls = set()  # Use a set to avoid duplicate links
    while len(video_urls) < max_urls:
        try:
            # Re-locate the video elements on each iteration to avoid stale references
            videos = driver.find_elements(By.XPATH, "//a[@id='video-title']")
            for video in videos:
                try:
                    href = video.get_attribute("href")
                    if href and "watch" in href:  # Ensure it's a valid video URL
                        video_urls.add(href)
                    if len(video_urls) >= max_urls:
                        break
                except StaleElementReferenceException:
                    continue  # Skip stale elements and re-iterate

            # Scroll down to load more videos
            driver.execute_script("window.scrollBy(0, 1000);")

        except StaleElementReferenceException:
            # If a stale element exception occurs, log and continue
            print("Encountered a stale element. Re-locating elements...")
            continue

    return list(video_urls)


    
def get_youtube_video_urls(driver, search_topic, max_urls):
   
    # Search for the given topic
    search_box = driver.find_element(By.NAME, "search_query")
    search_box.send_keys(search_topic)
    search_box.submit()
    time.sleep(5)  # Wait for results to load

    apply_filters(driver)

    # Scroll and collect video URLs
    video_urls = set()  # Use a set to avoid duplicate links
    while len(video_urls) < max_urls:
        # Find all video links on the current page
        videos = driver.find_elements(By.XPATH, "//a[@id='video-title']")
        for video in videos:
            href = video.get_attribute("href")
            if href and "watch" in href:  # Ensure it's a valid video URL
                video_urls.add(href)
            if len(video_urls) >= max_urls:
                break

        # Scroll down to load more videos
        driver.execute_script("window.scrollBy(0, 100);")
       # time.sleep(random.uniform(1, 3))  # Random delay to mimic human behavior

    try:
        clear_button = driver.find_element(By.ID, "search-clear-button")
        clear_button.click()
        time.sleep(2)  # Wait for the page to update
    except Exception as e:
        print("Clear button not found or could not be clicked:", e)

    return list(video_urls)

In [ ]:
# Search for a general topic to load videos (or stay on the homepage)
search_topic_list = ["Courses", "Tech Videos", "Reviews" ,"Podcasts","Cooking Videos", "Interviews with Celebs"]


final_df = pd.DataFrame()

for search_topic in search_topic_list:

    video_urls_list = get_youtube_video_urls1(driver,search_topic,100)
    final_df[search_topic] = pd.Series(video_urls_list)


# Save all URLs to a single Excel file
output_file = "youtube_video_urls.xlsx"
final_df.to_excel(output_file, index=False)


# Close the browser
driver.quit()


print(f"Successfully saved  YouTube video URLs to {output_file}.")

Parsing URL to get Video IDs

In [ ]:
import pandas as pd
from urllib.parse import urlparse, parse_qs

# Load the Excel file
file_path = '/Users/mandy/Desktop/NLP/Project/youtube_video_urls.xlsx'
df = pd.read_excel(file_path)

# Function to extract YouTube video ID from a URL
def extract_video_id(youtube_url):
    parsed_url = urlparse(youtube_url)
    query_params = parse_qs(parsed_url.query)
    video_id = query_params.get("v")
    return video_id[0] if video_id else None

# Apply the function to each column in the DataFrame (excluding headings)
for column in df.columns:
    df[column] = df[column].apply(lambda url: extract_video_id(url) if pd.notnull(url) else None)

# Save the updated DataFrame to a new Excel file
output_file = '/Users/mandy/Desktop/NLP/Project/youtube_video_ids.xlsx'
df.to_excel(output_file, index=False)

output_file

To get transcripts form video IDs

In [32]:
from youtube_transcript_api import YouTubeTranscriptApi

# Function to get the transcript for a YouTube video with fallback to 'en-IN'
def get_transcript_with_fallback(video_id):
    try:
        # Attempt to fetch the transcript with default language preferences
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        return transcript
    except Exception as e:
        # Check if 'en-IN' is available
        try:
            available_transcripts = YouTubeTranscriptApi.list_transcripts(video_id)
            if 'en-IN' in [t.language_code for t in available_transcripts]:
                # Fetch the 'en-IN' transcript
                return available_transcripts.find_transcript(['en-IN']).fetch()
            else:
                # If 'en-IN' is not available, re-raise the original exception
                raise Exception(f"Error fetching transcript for video ID {video_id}: {e}")
        except Exception as inner_e:
            raise Exception(f"Error fetching transcript for video ID {video_id}: {inner_e}")

# Function to generate timestamps from a transcript
def generate_timestamps(transcript):
    timestamps = []
    if isinstance(transcript, list):  # Ensure transcript is valid
        for item in transcript:
            timestamps.append(f"{item['start']} - {item['text']}")
    else:
        timestamps.append(transcript)  # If there's an error, log the error message
    return timestamps


# Load the Excel file containing the video IDs
file_path = '/Users/mandy/Desktop/NLP/Project/Sample Chapters.xlsx'
#file_path = '/Users/mandy/Desktop/NLP/Project/youtube_video_urls.xlsx'
df = pd.read_excel(file_path)

# Extract video IDs from column B starting from row 2
video_ids = df.iloc[:, 1].dropna().tolist()  # Assuming column B has video IDs

#print(video_ids)

timestamps_data = []

for video_id in video_ids:
    try:
        transcript = get_transcript_with_fallback(video_id)
        timestamps = generate_timestamps(transcript)
        timestamps_data.append({"Video ID": video_id, "Timestamps": "\n".join(timestamps)})
    except Exception as e:
        timestamps_data.append({"Video ID": video_id, "Timestamps": str(e)})

# Save the timestamps to a DataFrame and then to an Excel file
timestamps_df = pd.DataFrame(timestamps_data)

# Save to Excel
output_file = '/Users/mandy/Desktop/NLP/Project/youtube_video_timestamps_with_fallback.xlsx'
timestamps_df.to_excel(output_file, index=False)

print(f"Timestamps saved to {output_file}")

Timestamps saved to /Users/mandy/Desktop/NLP/Project/youtube_video_timestamps_with_fallback.xlsx


In [ ]:
# Path to the file containing timestamps and errors
file_path = '/Users/mandy/Desktop/NLP/Project/youtube_video_timestamps_with_fallback.xlsx'

# Load the file into a DataFrame
df = pd.read_excel(file_path)

# Count rows where the "Timestamps" column contains error messages
error_count = df['Timestamps'].str.contains("Error fetching transcript", na=False).sum()

# Display the number of errors
print(f"Number of errors in the file: {error_count}")

In [ ]:
cleaned_df = df[~df['Timestamps'].str.contains("Error fetching transcript", na=False)]

# Save the cleaned DataFrame to a new Excel file
cleaned_file_path = '/Users/mandy/Desktop/NLP/Project/youtube_video_timestamps_cleaned.xlsx'
cleaned_df.to_excel(cleaned_file_path, index=False)

print(f"Cleaned file saved to: {cleaned_file_path}")

# Count rows where the "Timestamps" column contains error messages
error_count = cleaned_df['Timestamps'].str.contains("Error fetching transcript", na=False).sum()

# Display the number of errors
print(f"Number of errors in the file: {error_count}")

ChatGPT API integration

In [ ]:
pip install openai

In [ ]:
import openai

from dotenv import dotenv_values

# Load the .env file as a dictionary
config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")

api_key = config["OPENAI_API_KEY"]

# Set your API key
openai.api_key = api_key

print("hello")

# Define a function to interact with ChatGPT
def chat_with_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Specify the model, e.g., "gpt-3.5-turbo" or "gpt-4"
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']



In [ ]:
import openai

from dotenv import dotenv_values

# Load the .env file as a dictionary
config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")

api_key = config["OPENAI_API_KEY"]
# Set your API key
openai.api_key = config["OPENAI_API_KEY"]

#print(config["OPENAI_API_KEY"])



# Function to interact with ChatGPT
def chat_with_gpt(prompt):
    # Use the updated API method
    response = openai.chat.completions.create(
        model="gpt-4",  # Replace with "gpt-3.5-turbo" if using that model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Example usage
if __name__ == "__main__":
    user_prompt = "What is the capital of France?"
    response = chat_with_gpt(user_prompt)
    print("ChatGPT says:", response)


In [40]:
def process_transcripts_with_chatgpt(video_id, transcripts):
    """Processes transcripts using ChatGPT API to generate video chapters."""
    prompt = f"""
    Create video chapters with timestamps from the following transcript for video ID {video_id}:
    {transcripts}
    Output format:
    [timestamp] - [Chapter Title]
    Without any description, at most 10 or less Top most titles(less than 7 Words) 
    """
    response =openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for processing video transcripts."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [41]:
import time
import pandas as pd


def main():
    input_file = '/Users/mandy/Desktop/NLP/Project/Sample_Dataset_10.xlsx'
    output_file = '/Users/mandy/Desktop/NLP/Project/video_chapters_generated.xlsx'

    print("Reading transcript data...")
    transcript_data = pd.read_excel(input_file)

    chapters = []
    print("Processing each video's transcripts...")
    for index, row in transcript_data.iterrows():
        video_id = row['Video ID']
        transcripts = row['Transcripts']
        
        # Monitor token usage and ensure compliance with rate limits
        try:
            print(f"Processing video ID: {video_id}")
            chapter = process_transcripts_with_chatgpt(video_id, transcripts)
            print(chapter)
            chapters.append({"Video ID": video_id, "Video Chapters": chapter})
        except openai.error.RateLimitError:
            print("Rate limit hit. Pausing for 60 seconds...")
            time.sleep(60)  # Pause for a minute before retrying
            chapter = process_transcripts_with_chatgpt(video_id, transcripts)
            chapters.append({"Video ID": video_id, "Video Chapters": chapter})
        except Exception as e:
            print(f"Error processing video ID {video_id}: {e}")
            chapters.append({"Video ID": video_id, "Video Chapters": "Error generating chapters"})
        
        # Respect rate limits: 500 RPM or 1 request every ~0.12 seconds
        time.sleep(0.15)  # Adds a slight delay between requests

    print("Saving generated chapters to file...")
    chapters_df = pd.DataFrame(chapters)
    chapters_df.to_excel(output_file, index=False)
    print(f"Video chapters saved to {output_file}")

if __name__ == "__main__":
    main()

Reading transcript data...
Processing each video's transcripts...
Processing video ID: L14GC1UFWcE
1. [0.0] - Introduction to Mental Health Skills
2. [6.24] - Exercise and Mindfulness for Anxiety
3. [11.34] - Understanding Clinical Depression
4. [17.04] - Depression as a Biological Trait
5. [29.1] - Misconceptions About Mental Health
6. [46.38] - Medical Model in Mental Health
7. [96.72] - Mental Health Treatment Success
8. [162.54] - Story of the Captive Elephant
9. [235.02] - Misunderstanding Medical Models
10. [335.4] - Biopsychosocial Model for Depression
11. [557.16] - Over-Identifying with Diagnosis
12. [685.02] - Overcoming Over-Identification
13. [727.98] - Course Information and Conclusion

Processing video ID: w70BkCqgyyI
[0.089] - Introduction to European History
[6.65] - The Black Death and Wars
[16.15] - Ice Age, Witch Hunts, Slave Trade
[22.19] - Turning to the Scientific Revolution
[36.93] - Persecution of Scientists
[53.09] - Introduction to the Scientific Revolution
[9

To Find tokens of query

In [ ]:
pip install transformers

In [ ]:
from transformers import GPT2Tokenizer

# Load a GPT-2 tokenizer (compatible with GPT-3 and GPT-4 token estimation)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define the prompt and transcript
video_id = "example_video_id"
transcript = """
0.0 - hey friends I'm Aliena and welcome back to my channel so in the past four months or so
......
to dedicate 30 minutes every day that is my only  
424.92 - strategy of finishing all these certificates 
in this year and I do not regret taking any  
430.5 - of these courses if you like my video and want to 
learn more about how I learn stuff then subscribe
435.96 - to my channel turn on all notifications and follow me on Instagram @thealienacai cheers
"""

# Define the full prompt with the transcript
prompt = f"""
Create video chapters with timestamps from the following transcript for video ID {video_id}:
{transcript}
Output format:
[timestamp] - [Chapter Title]
"""

# Tokenize and calculate the token count
tokens = tokenizer.encode(prompt)
token_count = len(tokens)

print(f"Token Count: {token_count}")

Chatgpt 

In [1]:
import time
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from dotenv import dotenv_values


config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")

email = config["Email"]
password = config["Password"]

driver = webdriver.Firefox()
driver.get('https://chat.openai.com/auth/login')
time.sleep(3) 

try:
    driver.get('https://chat.openai.com/auth/login')
    time.sleep(3) 


   
finally:
   print("DONE!")


DONE!


Preparing the dataset for fine tuning

In [ ]:
import pandas as pd

# Load the Excel file
input_file_path = "/Users/mandy/Desktop/NLP/Project/Training data.xlsx"  # Replace with your file path
output_file_path = "/Users/mandy/Desktop/NLP/Project/Training_converted_excel_file.xlsx"  # Replace with the desired output file path


# Read the input data
input_data = pd.read_excel(input_file_path, header=None)

# Transform each cell in the DataFrame by replacing newlines with spaces
input_data = input_data.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)

# Save the transformed data to a new Excel file
input_data.to_excel(output_file_path, index=False, header=False)

print(f"Converted data has been saved to {output_file_path}.")

In [25]:
import pandas as pd

# Load the Excel file
file_path = '/Users/mandy/Desktop/NLP/Project/Training_converted_excel_file.xlsx'  # Replace with your file path
output_file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'
text_to_add = "Generate the timestamps by following these rules: 1) I need only titles and timestamps 2)Timestamps should be on the left before the title 3)Convert the timestamps to minutes, in transcripts they are in seconds (Example: 341.28 seconds = 5:41 (5 minutes and 41 seconds), only chapter them into 10 or fewer chapters(Does have to be 10, if it lasts less than 10 minutes long, generate less than 10 chapters) Transcripts:"  # The text to prepend

# Read the Excel file
df = pd.read_excel(file_path)

# Prepend the text to every element in the first column
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: f"{text_to_add}{x}" if pd.notnull(x) else x)

# Save the updated DataFrame back to Excel
df.to_excel(output_file_path, index=False)

print(f"Updated Excel file saved to {output_file_path}")

Updated Excel file saved to /Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx


In [26]:
import pandas as pd

# Load the Excel file
file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'  # Replace with your file path
output_file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'
text_to_add = "Timestamps: "  # The text to prepend

# Read the Excel file
df = pd.read_excel(file_path)

# Prepend the text to every element in the first column
df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: f"{text_to_add}{x}" if pd.notnull(x) else x)

# Save the updated DataFrame back to Excel
df.to_excel(output_file_path, index=False)

print(f"Updated Excel file saved to {output_file_path}")

Updated Excel file saved to /Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx


Dataset for Finetuning

In [3]:
import pandas as pd
import json

# Load the dataset
file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'  # Update this path to your actual file location
data = pd.ExcelFile(file_path)

# Load the first sheet
sheet_data = data.parse(data.sheet_names[0])

# Fill missing values with empty strings and ensure all columns are strings
sheet_data['Transcripts'] = sheet_data['Transcripts'].fillna('').astype(str)
sheet_data['Chapters'] = sheet_data['Chapters'].fillna('').astype(str)

# Function to convert the dataset into the desired JSON format
def convert_to_json_format(dataframe):
    json_output = {"messages": []}
    for _, row in dataframe.iterrows():
        # Add user role and content (Prompt)
        json_output["messages"].append({
            "role": "user",
            "content": row['Transcripts']
        })
        # Add assistant role and content (Content)
        json_output["messages"].append({
            "role": "assistant",
            "content": row['Chapters'].replace('\n', ' ').strip()  # Remove newlines and extra spaces
        })
    return json_output

# Convert the data into the desired JSON format
formatted_json = convert_to_json_format(sheet_data)

# Save the formatted JSON to a file
output_file_path = '/Users/mandy/Desktop/NLP/Project/mistral_training_data.json'  # Update this path to your desired output location
#with open(output_file_path, 'w') as json_file:
   # json.dump(formatted_json, json_file, indent=2)

print(f"JSON file has been saved to {output_file_path}")


JSON file has been saved to /Users/mandy/Desktop/NLP/Project/mistral_training_data.json


In [ ]:
import pandas as pd
import json

# Load the dataset
file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'  # Update this path to your actual file location
data = pd.ExcelFile(file_path)

sheet_data = data.parse(data.sheet_names[0])

# Fill missing values with empty strings and ensure all columns are strings
sheet_data['Transcripts'] = sheet_data['Transcripts'].fillna('').astype(str)
sheet_data['Chapters'] = sheet_data['Chapters'].fillna('').astype(str)

# Define the JSONL conversion function
def convert_to_jsonl(dataframe, output_file):
    with open(output_file, 'w') as f:
        for _, row in dataframe.iterrows():
            # Construct the JSON structure for each row
            record = {
                "messages": [
                    {
                        "role": "user",
                        "content": row['Transcripts']  # Replace with the column name for the user prompt
                    },
                    {
                        "role": "assistant",
                        "content": row['Chapters']  # Replace with the column name for the assistant response
                    }
                ]
            }
            # Write each record to the file as a JSON line
            f.write(json.dumps(record) + '\n')

# Define the output file path
output_jsonl_file = "mistral_training_data.jsonl"

# Convert and save to JSONL
convert_to_jsonl(sheet_data, output_jsonl_file)

print(f"JSONL file saved to: {output_jsonl_file}")

In [38]:
import pandas as pd
import json

# Load the dataset
file_path = '/Users/mandy/Desktop/NLP/Project/Updated Training data.xlsx'  # Update this path to your actual file location
data = pd.ExcelFile(file_path)

# Load the first sheet
sheet_data = data.parse(data.sheet_names[0])

# Fill missing values with empty strings and ensure all columns are strings
sheet_data['Transcripts'] = sheet_data['Transcripts'].fillna('').astype(str)
sheet_data['Chapters'] = sheet_data['Chapters'].fillna('').astype(str)

# Function to convert the dataset into the desired JSON format
def convert_to_json_format(dataframe):
    json_output = {"messages": []}
    for _, row in dataframe.iterrows():
        # Add user role and content (Prompt)
        json_output["messages"].append({
            "role": "user",
            "content": row['Transcripts']
        })
        # Add assistant role and content (Content)
        json_output["messages"].append({
            "role": "assistant",
            "content": row['Chapters'].replace('\n', ' ').strip()  # Remove newlines and extra spaces
        })
    return json_output

# Convert the data into the desired JSON format
formatted_json = convert_to_json_format(sheet_data)



# Save the formatted JSON to a file without escaping Unicode characters
output_file_path = '/Users/mandy/Desktop/NLP/Project/mistral_training_data.json'  # Update this path to your desired output location
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(formatted_json, json_file, indent=2, ensure_ascii=False)  # ensure_ascii=False keeps Unicode characters readable

print(f"JSON file has been saved to {output_file_path}")

JSON file has been saved to /Users/mandy/Desktop/NLP/Project/mistral_training_data.json


Convert it to JsonL file

In [4]:
import json

# Load the provided JSON file
input_file_path = '/Users/mandy/Desktop/NLP/Project/mistral_training_data.json'
output_file_path = '/Users/mandy/Desktop/NLP/Project/mistral_training_data.jsonl'

# Open and load the JSON data
with open(input_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Write the data to JSONL format
with open(output_file_path, 'w', encoding='utf-8') as jsonl_file:
    if isinstance(data, list):  # If the JSON contains a list of objects
        for record in data:
            #jsonl_file.write(json.dumps(record, ensure_ascii=False) + '\n')
            jsonl_file.write(json.dumps(record) + '\n')
    elif isinstance(data, dict):  # If the JSON is a dictionary
        #jsonl_file.write(json.dumps(data, ensure_ascii=False) + '\n')
        jsonl_file.write(json.dumps(data) + '\n')

output_file_path


'/Users/mandy/Desktop/NLP/Project/mistral_training_data.jsonl'

In [6]:
pip install Mistral

python(45693) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install mistralai

python(45715) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 4.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from mistralai import Mistral


from dotenv import dotenv_values

# Load the .env file as a dictionary
config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")

api_key = config["MISTRAL_API_KEY"]
model = "ft:open-mistral-7b:a620db77:20241202:a35bfbff"

client = Mistral(api_key=api_key)


chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content":"""Generate the timestamps by following these rules: 1) I need only titles and timestamps 2)Timestamps should be on the left before the title 3)Convert the timestamps to minutes, in transcripts they are in seconds (Example: 341.28 seconds = 5:41 (5 minutes and 41 seconds), only chapter them into 10 or fewer chapters(Does have to be 10, if it lasts less than 10 minutes long, generate less than 10 chapters) Transcripts:0.0 - hey friends I'm Aliena and welcome back to  my channel so in the past four months or so   5.52 - I finished another Google certificate course  and this one it is the digital marketing and   11.22 - e-commerce certificate I took this course because  I work in the e-commerce industry so for those of   17.04 - you who consider taking the course I want to  break down what is exactly in these courses   21.9 - and I want to let you know whether I think it  is worth it or not first of all this is a large   27.96 - certificate course it's not just one certificate  it also has seven smaller certificates so there   33.48 - are seven courses in total you can also just take  one course and then get that certificate and it's   39.9 - not a course that you can finish in one sitting  you need to plan to spend months on the courses   45.96 - to finish everything there will be videos readings  quizzes and activities you'll be doing throughout   53.46 - the courses if you can pay for the course I will  recommend you to pay for the Coursera plus plan   60.18 - because that one you can not only take this  course you can take also any other course in   65.76 - coursera whenever there's a sale going on I'll be  putting the link to my description if you cannot   72.3 - afford the course no worries at all you can audit  it for free or you can apply for a financial aid   79.44 - let me quickly break down each course for you so  that you don't have to waste time thinking about   84.54 - which one should I take and which one should  I skip course one is about the fundamentals   89.76 - of digital marketing and e-commerce and by the  way digital marketing and e-commerce are two   94.86 - different things but this course just decide to  put them together and teach you at the same time   99.84 - so course one I would suggest everyone to take  because that's the fundamental it will run you   104.88 - through the digital marketing funnel it will  explain to you what is the difference between   109.62 - paid media versus owned media versus  earned media different types of ads   114.24 - SEO and then types of marketing campaign for  course 2 it's more heavily focused on SEO and   121.98 - ads so if you work in the industry related to  advertising or SEO this could be good for you   129.78 - course 3 is about social media marketing so if  you want to learn about social media marketing   134.28 - or do anything social media marketing related  take this one it also covers ads on social media   140.04 - advertising people you can check out the later  part of this course the fourth course is all   145.2 - about email marketing so if you get a job that's  highly related to email marketing take this course   151.32 - or if you want to learn about how to use email  to grow your small business take this course   157.08 - course five is about marketing or marketing  analysis and a little bit about Google ads   163.62 - this course is slightly more technical you will  see a lot of data and dashboard so for those of   171.84 - you that like numbers data analysis take this  one course six is about starting an e-commerce   178.56 - store especially on Shopify and it also touches  a little bit on advertising so if you want to   184.56 - start a Shopify store yourself or if you want to  learn more about e-commerce* and ads take this   189.72 - course and finally course 7 is about customer  loyalty how to write a great resume so if you   197.34 - want to get a job take this course now regarding  what I liked about the course first of all the   205.26 - breadth of the course was mind-blowing I feel  like I learned so many different things because   210.42 - I've also taken other Coursera Google courses  right like the ux design one data analysis one   215.4 - and the PM course from Google those are more  specific designed for one type of job but this   222.12 - course brought your knowledge about the entire  industry I definitely feel like I've learned a lot   229.08 - and just like what Google always does the video  quality is really good I really enjoy looking at   236.34 - high quality video and short videos so most of  the things are like five minutes long it's just   242.34 - way better than listen to a lecture for three  hours so for that part I highly enjoyed another   250.32 - thing that Google and Coursera really do well is  that they'll bring interactive activity in this   256.14 - learning process so even if the video is just five  minutes long you'll still see questions popping   261.66 - up in the middle of the video that test your  knowledge on what you just saw and this course   266.34 - brings you a lot of great resources either it is  links to things that you can learn deeper on or   273.12 - it's just like layouts of different job titles in  this industry other than that there's also a lot   279.9 - of practice in the course Coursera and Google  always bring good activity good scenarios so   286.68 - make sure you read through those scenarios don't  skip the activity those are really really spot on   291.72 - now moving to what I disliked about the course  as I said before I think the course touches on   298.14 - many many things it kind of made me feel like the  knowledge is a little bit scattered so I almost   304.86 - will suggest you to take the first course and then  directly jump to the one that you want to learn   310.5 - the most and after that you can just take your  time and look through the other videos for example   316.26 - if you only want to learn about email marketing  I'm not sure whether going through all the details   321.72 - about like ads* dashboards Google ads will help  you too much another thing that I kind of disliked   328.8 - is that they mixed together digital marketing  and e-commerce too much that I forgot to think   333.42 - about oh there's actually a difference between  digital marketing and e-commerce I work in the   338.04 - e-commerce industry because I work for eBay but  at the same time me doing YouTube channel is some   344.46 - sort of digital marketing for myself and I somehow  want to separate the concept a bit more you know   350.94 - so when you're taking the course please keep in  mind that even though the brand digital marketing   355.56 - and e-commerce together it's still not entirely  the same and finally I want to say that I don't   362.22 - think this certificate will help you too much if  your goal is to start an e-commerce store yourself   366.84 - because this course is more on the knowledge side  of thing instead of the practice side of thing   372.9 - if you want to start an e-commerce store  you will have to import you will have to   377.52 - take photos check the price communicate  with the buyers and that I think really   383.22 - need practice instead of just pure knowledge  but if your goal is to work in the digital   389.82 - marketing or e-commerce industry or if  you already got a job in these industries   396.12 - I think this is a great place for you to be it  just brings you so much knowledge you can talk   400.14 - about in those interviews but again knowledge is  different from experience to get an actual job   407.22 - you still need to show you have experience just by  showing you have a certificate will almost never   412.62 - get you a job and final advice for every single  one of you the best way of finishing anything   419.34 - certificate course related* or learn anything is  to dedicate 30 minutes every day that is my only   424.92 - strategy of finishing all these certificates  in this year and I do not regret taking any   430.5 - of these courses if you like my video and want to  learn more about how I learn stuff then subscribe   435.96 - to my channel turn on all notifications and  follow me on Instagram @thealienacai cheers""",
        },
    ]
)
print(chat_response.choices[0].message.content)


Timestamps: 0:00 - Introduction 0:10 - Introduction to Course 0:30 - Course Overview 1:00 - Course Structure and 
Format 1:40 - Coursera Plus and Financial Aid Options 2:10 - Breaking Down the Courses 2:40 - Course 1: 
Fundamentals of Digital Marketing and E-Commerce 3:30 - Course 2: SEO and Advertising 4:10 - Course 3: Social Media
Marketing 4:50 - Course 4: Email Marketing 5:30 - Course 5: Marketing Analysis and Google Ads 6:10 - Course 6: 
Starting an E-Commerce Store 6:50 - Course 7: Customer Loyalty and Resumes 7:30 - Review of Course Assets and 
Activities 8:10 - Criticisms and Suggestions 8:50 - Certificate Value for E-Commerce Store Owners 9:30 - Final 
Advice for Course Completion

Generate the timestamps by following these rules: 1) I need only titles and timestamps 2)Timestamps should be on the left before the title 3)Convert the timestamps to minutes, in transcripts they are in seconds (Example: 341.28 seconds = 5:41 (5 minutes and 41 seconds), only chapter them into 10 or fewer chapters(Does have to be 10, if it lasts less than 10 minutes long, generate less than 10 chapters) Transcripts:0.0 - hey friends I'm Aliena and welcome back to  my channel so in the past four months or so   5.52 - I finished another Google certificate course  and this one it is the digital marketing and   11.22 - e-commerce certificate I took this course because  I work in the e-commerce industry so for those of   17.04 - you who consider taking the course I want to  break down what is exactly in these courses   21.9 - and I want to let you know whether I think it  is worth it or not first of all this is a large   27.96 - certificate course it's not just one certificate  it also has seven smaller certificates so there   33.48 - are seven courses in total you can also just take  one course and then get that certificate and it's   39.9 - not a course that you can finish in one sitting  you need to plan to spend months on the courses   45.96 - to finish everything there will be videos readings  quizzes and activities you'll be doing throughout   53.46 - the courses if you can pay for the course I will  recommend you to pay for the Coursera plus plan   60.18 - because that one you can not only take this  course you can take also any other course in   65.76 - coursera whenever there's a sale going on I'll be  putting the link to my description if you cannot   72.3 - afford the course no worries at all you can audit  it for free or you can apply for a financial aid   79.44 - let me quickly break down each course for you so  that you don't have to waste time thinking about   84.54 - which one should I take and which one should  I skip course one is about the fundamentals   89.76 - of digital marketing and e-commerce and by the  way digital marketing and e-commerce are two   94.86 - different things but this course just decide to  put them together and teach you at the same time   99.84 - so course one I would suggest everyone to take  because that's the fundamental it will run you   104.88 - through the digital marketing funnel it will  explain to you what is the difference between   109.62 - paid media versus owned media versus  earned media different types of ads   114.24 - SEO and then types of marketing campaign for  course 2 it's more heavily focused on SEO and   121.98 - ads so if you work in the industry related to  advertising or SEO this could be good for you   129.78 - course 3 is about social media marketing so if  you want to learn about social media marketing   134.28 - or do anything social media marketing related  take this one it also covers ads on social media   140.04 - advertising people you can check out the later  part of this course the fourth course is all   145.2 - about email marketing so if you get a job that's  highly related to email marketing take this course   151.32 - or if you want to learn about how to use email  to grow your small business take this course   157.08 - course five is about marketing or marketing  analysis and a little bit about Google ads   163.62 - this course is slightly more technical you will  see a lot of data and dashboard so for those of   171.84 - you that like numbers data analysis take this  one course six is about starting an e-commerce   178.56 - store especially on Shopify and it also touches  a little bit on advertising so if you want to   184.56 - start a Shopify store yourself or if you want to  learn more about e-commerce* and ads take this   189.72 - course and finally course 7 is about customer  loyalty how to write a great resume so if you   197.34 - want to get a job take this course now regarding  what I liked about the course first of all the   205.26 - breadth of the course was mind-blowing I feel  like I learned so many different things because   210.42 - I've also taken other Coursera Google courses  right like the ux design one data analysis one   215.4 - and the PM course from Google those are more  specific designed for one type of job but this   222.12 - course brought your knowledge about the entire  industry I definitely feel like I've learned a lot   229.08 - and just like what Google always does the video  quality is really good I really enjoy looking at   236.34 - high quality video and short videos so most of  the things are like five minutes long it's just   242.34 - way better than listen to a lecture for three  hours so for that part I highly enjoyed another   250.32 - thing that Google and Coursera really do well is  that they'll bring interactive activity in this   256.14 - learning process so even if the video is just five  minutes long you'll still see questions popping   261.66 - up in the middle of the video that test your  knowledge on what you just saw and this course   266.34 - brings you a lot of great resources either it is  links to things that you can learn deeper on or   273.12 - it's just like layouts of different job titles in  this industry other than that there's also a lot   279.9 - of practice in the course Coursera and Google  always bring good activity good scenarios so   286.68 - make sure you read through those scenarios don't  skip the activity those are really really spot on   291.72 - now moving to what I disliked about the course  as I said before I think the course touches on   298.14 - many many things it kind of made me feel like the  knowledge is a little bit scattered so I almost   304.86 - will suggest you to take the first course and then  directly jump to the one that you want to learn   310.5 - the most and after that you can just take your  time and look through the other videos for example   316.26 - if you only want to learn about email marketing  I'm not sure whether going through all the details   321.72 - about like ads* dashboards Google ads will help  you too much another thing that I kind of disliked   328.8 - is that they mixed together digital marketing  and e-commerce too much that I forgot to think   333.42 - about oh there's actually a difference between  digital marketing and e-commerce I work in the   338.04 - e-commerce industry because I work for eBay but  at the same time me doing YouTube channel is some   344.46 - sort of digital marketing for myself and I somehow  want to separate the concept a bit more you know   350.94 - so when you're taking the course please keep in  mind that even though the brand digital marketing   355.56 - and e-commerce together it's still not entirely  the same and finally I want to say that I don't   362.22 - think this certificate will help you too much if  your goal is to start an e-commerce store yourself   366.84 - because this course is more on the knowledge side  of thing instead of the practice side of thing   372.9 - if you want to start an e-commerce store  you will have to import you will have to   377.52 - take photos check the price communicate  with the buyers and that I think really   383.22 - need practice instead of just pure knowledge  but if your goal is to work in the digital   389.82 - marketing or e-commerce industry or if  you already got a job in these industries   396.12 - I think this is a great place for you to be it  just brings you so much knowledge you can talk   400.14 - about in those interviews but again knowledge is  different from experience to get an actual job   407.22 - you still need to show you have experience just by  showing you have a certificate will almost never   412.62 - get you a job and final advice for every single  one of you the best way of finishing anything   419.34 - certificate course related* or learn anything is  to dedicate 30 minutes every day that is my only   424.92 - strategy of finishing all these certificates  in this year and I do not regret taking any   430.5 - of these courses if you like my video and want to  learn more about how I learn stuff then subscribe   435.96 - to my channel turn on all notifications and  follow me on Instagram @thealienacai cheers

In [11]:
import os
from rich import print
from mistralai.client import MistralClient

import gradio as gr

from dotenv import dotenv_values


# Load the .env file as a dictionary
config = dotenv_values("/Users/mandy/Desktop/NLP/Project/Key.env")
#Function to get chat response

def generate_response(user_input):
    """
    Generate a response from the Mistral API based on user input.
    
    Args:
        user_input (str): The input text to send to the API.
    
    Returns:
        str: The response from the Mistral API.
    """
    # Fetch API key from the config
    api_key = config["MISTRAL_API_KEY"]
    
    # Define the model ID
    model = "ft:open-mistral-7b:a620db77:20241202:a35bfbff"
    model1 = "open-mistral-7b"

    # Initialize the Mistral client
    client = Mistral(api_key=api_key)

    # Call the Mistral API with user input
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "user",
                "content": user_input,  # Use the user-provided input here
            },
        ]
    )

    # Return the API response content
    return chat_response.choices[0].message.content


    #Gradio Interface
iface = gr.Interface(
fn=generate_response,
inputs="text",
outputs="text", 
title="Mistral A Chatbot",
description="Ask the chatbot any question.",
)
iface.launch()




Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [1]:
from mistralai import Mistral

model = "ft:open-mistral-7b:a620db77:20241202:a35bfbff"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "Generate the timestamps by following these rules: 1) I need only titles and timestamps 2)Timestamps should be on the left before the title 3)Convert the timestamps to minutes, in transcripts they are in seconds (Example: 341.28 seconds = 5:41 (5 minutes and 41 seconds), only chapter them into 10 or fewer chapters(Does have to be 10, if it lasts less than 10 minutes long, generate less than 10 chapters) Transcripts:0.0 - hey friends I'm Aliena and welcome back to  my channel so in the past four months or so   5.52 - I finished another Google certificate course  and this one it is the digital marketing and   11.22 - e-commerce certificate I took this course because  I work in the e-commerce industry so for those of   17.04 - you who consider taking the course I want to  break down what is exactly in these courses   21.9 - and I want to let you know whether I think it  is worth it or not first of all this is a large   27.96 - certificate course it's not just one certificate  it also has seven smaller certificates so there   33.48 - are seven courses in total you can also just take  one course and then get that certificate and it's   39.9 - not a course that you can finish in one sitting  you need to plan to spend months on the courses   45.96 - to finish everything there will be videos readings  quizzes and activities you'll be doing throughout   53.46 - the courses if you can pay for the course I will  recommend you to pay for the Coursera plus plan   60.18 - because that one you can not only take this  course you can take also any other course in   65.76 - coursera whenever there's a sale going on I'll be  putting the link to my description if you cannot   72.3 - afford the course no worries at all you can audit  it for free or you can apply for a financial aid   79.44 - let me quickly break down each course for you so  that you don't have to waste time thinking about   84.54 - which one should I take and which one should  I skip course one is about the fundamentals   89.76 - of digital marketing and e-commerce and by the  way digital marketing and e-commerce are two   94.86 - different things but this course just decide to  put them together and teach you at the same time   99.84 - so course one I would suggest everyone to take  because that's the fundamental it will run you   104.88 - through the digital marketing funnel it will  explain to you what is the difference between   109.62 - paid media versus owned media versus  earned media different types of ads   114.24 - SEO and then types of marketing campaign for  course 2 it's more heavily focused on SEO and   121.98 - ads so if you work in the industry related to  advertising or SEO this could be good for you   129.78 - course 3 is about social media marketing so if  you want to learn about social media marketing   134.28 - or do anything social media marketing related  take this one it also covers ads on social media   140.04 - advertising people you can check out the later  part of this course the fourth course is all   145.2 - about email marketing so if you get a job that's  highly related to email marketing take this course   151.32 - or if you want to learn about how to use email  to grow your small business take this course   157.08 - course five is about marketing or marketing  analysis and a little bit about Google ads   163.62 - this course is slightly more technical you will  see a lot of data and dashboard so for those of   171.84 - you that like numbers data analysis take this  one course six is about starting an e-commerce   178.56 - store especially on Shopify and it also touches  a little bit on advertising so if you want to   184.56 - start a Shopify store yourself or if you want to  learn more about e-commerce* and ads take this   189.72 - course and finally course 7 is about customer  loyalty how to write a great resume so if you   197.34 - want to get a job take this course now regarding  what I liked about the course first of all the   205.26 - breadth of the course was mind-blowing I feel  like I learned so many different things because   210.42 - I've also taken other Coursera Google courses  right like the ux design one data analysis one   215.4 - and the PM course from Google those are more  specific designed for one type of job but this   222.12 - course brought your knowledge about the entire  industry I definitely feel like I've learned a lot   229.08 - and just like what Google always does the video  quality is really good I really enjoy looking at   236.34 - high quality video and short videos so most of  the things are like five minutes long it's just   242.34 - way better than listen to a lecture for three  hours so for that part I highly enjoyed another   250.32 - thing that Google and Coursera really do well is  that they'll bring interactive activity in this   256.14 - learning process so even if the video is just five  minutes long you'll still see questions popping   261.66 - up in the middle of the video that test your  knowledge on what you just saw and this course   266.34 - brings you a lot of great resources either it is  links to things that you can learn deeper on or   273.12 - it's just like layouts of different job titles in  this industry other than that there's also a lot   279.9 - of practice in the course Coursera and Google  always bring good activity good scenarios so   286.68 - make sure you read through those scenarios don't  skip the activity those are really really spot on   291.72 - now moving to what I disliked about the course  as I said before I think the course touches on   298.14 - many many things it kind of made me feel like the  knowledge is a little bit scattered so I almost   304.86 - will suggest you to take the first course and then  directly jump to the one that you want to learn   310.5 - the most and after that you can just take your  time and look through the other videos for example   316.26 - if you only want to learn about email marketing  I'm not sure whether going through all the details   321.72 - about like ads* dashboards Google ads will help  you too much another thing that I kind of disliked   328.8 - is that they mixed together digital marketing  and e-commerce too much that I forgot to think   333.42 - about oh there's actually a difference between  digital marketing and e-commerce I work in the   338.04 - e-commerce industry because I work for eBay but  at the same time me doing YouTube channel is some   344.46 - sort of digital marketing for myself and I somehow  want to separate the concept a bit more you know   350.94 - so when you're taking the course please keep in  mind that even though the brand digital marketing   355.56 - and e-commerce together it's still not entirely  the same and finally I want to say that I don't   362.22 - think this certificate will help you too much if  your goal is to start an e-commerce store yourself   366.84 - because this course is more on the knowledge side  of thing instead of the practice side of thing   372.9 - if you want to start an e-commerce store  you will have to import you will have to   377.52 - take photos check the price communicate  with the buyers and that I think really   383.22 - need practice instead of just pure knowledge  but if your goal is to work in the digital   389.82 - marketing or e-commerce industry or if  you already got a job in these industries   396.12 - I think this is a great place for you to be it  just brings you so much knowledge you can talk   400.14 - about in those interviews but again knowledge is  different from experience to get an actual job   407.22 - you still need to show you have experience just by  showing you have a certificate will almost never   412.62 - get you a job and final advice for every single  one of you the best way of finishing anything   419.34 - certificate course related* or learn anything is  to dedicate 30 minutes every day that is my only   424.92 - strategy of finishing all these certificates  in this year and I do not regret taking any   430.5 - of these courses if you like my video and want to  learn more about how I learn stuff then subscribe   435.96 - to my channel turn on all notifications and  follow me on Instagram @thealienacai cheers",
        },
    ]
)
print(chat_response.choices[0].message.content)

Timestamps: 0:00 - Introduction and Course Overview 0:10 - Choosing the Right Course 1:00 - Course Breakdown and Recommendations 2:00 - Learning and Resources 3:00 - Course Criticisms and Advice 4:00 - Final Advice and Strategy
